# Piotrek Magiczny Parser v.0.1

## Imports

In [1459]:
import glob, requests
import pandas as pd
import numpy as np
from datetime import datetime

# Functions

In [1460]:
def create_set_dictionary() -> dict:
    request = requests.get("https://api.scryfall.com/sets/")
    response: list[dict] = request.json()["data"]

    set_dict: dict = [
        {"code": element["code"], "name": element["name"]} for element in response
    ]
    return set_dict

In [1461]:
def cleanse_set_name(set_name: str) -> str:
    set_name_cleansed: str = set_name

    if "Battle for Baldur's Gate" in set_name:
        set_name_cleansed = "Commander Legends: Battle for Baldur's Gate"
    if "Commander 2013" in set_name:
        set_name_cleansed = "Commander 2013 Edition"
    if "Forgotten Realms Variants" in set_name:
        set_name_cleansed = "Adventures in the Forgotten Realms"
    if "Ikoria Variants" in set_name:
        set_name_cleansed = "Ikoria: Lair of Behemoths"
    if "Mystery Booster Playtest Cards 2021" in set_name:
        set_name_cleansed = "Mystery Booster Playtest Cards"
    if "Mystical Archive" in set_name:
        set_name_cleansed = "Strixhaven Mystical Archive"
    if "Promo Pack: Forgotten Realms" in set_name:
        set_name_cleansed = "Adventures in the Forgotten Realms"
    if "Promo Pack: Ikoria" in set_name:
        set_name_cleansed = "Ikoria: Lair of Behemoths"
    if "Promo Pack: Innistrad Midnight Hunt" in set_name:
        set_name_cleansed = "Innistrad: Midnight Hunt"
    if "Promo Pack: Theros Beyond Death" in set_name:
        set_name_cleansed = "Theros Beyond Death"
    if "Promo Pack: Throne of Eldraine" in set_name:
        set_name_cleansed = "Throne of Eldraine"
    if "Promo Pack: Zendikar Rising" in set_name:
        set_name_cleansed = "Zendikar Rising"
    if "Ravnica Allegiance Guild Kits" in set_name:
        set_name_cleansed = "RNA Guild Kit"
    if "Store Championship Promos" in set_name:
        set_name_cleansed = "Wizards Play Network 2021"
    if "Strixhaven" in set_name and not "Mystical Archive" in set_name:
        set_name_cleansed = "Strixhaven: School of Mages"
    if "Time Spiral Remastered (Timeshifted)" in set_name:
        set_name_cleansed = "Time Spiral Remastered"
    if "Magic 2015 Core Set" in set_name:
        set_name_cleansed = "Magic 2015"
    
    return set_name_cleansed.replace(" Variants", "")

In [1462]:
def find_correct_set(set_name: set, set_dicts: list[set]) -> str:
    correct_set: str = set_name
    try:
        correct_set = [ele for ele in set_dicts if ele['name'].replace(" Variants", "") == cleanse_set_name(set_name)][0]['code']
    except IndexError:
        correct_set = 'NOT FOUND'

    return correct_set

In [1463]:
def cleanse_card_name(card_name: str) -> str:
    if "(" not in card_name:
        return card_name
    
    result: str = [ele for ele in card_name.split(" (")][0]

    return result

In [1464]:
def decide_if_land(card_name: str) -> bool:
    lands: list = ['plains', 'island', 'swamp', 'mountain', 'forest']
    result: bool = True if card_name.lower() in lands else False
        
    return result

In [1465]:
def handle_data_exceptions(row, index, output_df) -> None:
    data: list[dict] = exceptions_df.to_dict('records')

    for record in data:
        if record['Old Name']:
            if row['Name'] != record['Old Name']:
                continue
        if record['Old CN']:
            if str(row['Number']) != str(record['Old CN']):
                continue
        if record['Old Set']:
            if row['Set'] != record['Old Set']:
                continue
        
        if record['New Name']:
            output_df.at[index, 'Name'] = record['New Name']
        
        if record['New CN']:
            output_df.at[index, 'Collector Number'] = str(record['New CN'])
        
        if record['New Set']:
            output_df.at[index, 'Set'] = record['New Set']
        
        if record['To Delete']:
            output_df.drop(index, inplace=True)



## Constants

In [1466]:
INPUT_FOLDER_PATH: str = glob.glob("./input/*")[0]
OUTPUT_FOLDER_PATH: str = f'./output/Aktualna_Kolekcja_{str(datetime.now()).split(".")[0].replace(" ", "").replace(":", "").replace("-", "")}.csv'
EXCEPTIONS_FILE: str = "./exceptions.xlsx"

## Inits

In [1467]:

df: pd.DataFrame = pd.read_csv(INPUT_FOLDER_PATH)
sets: dict = create_set_dictionary()
exceptions_df: pd.DataFrame = pd.read_excel(EXCEPTIONS_FILE).replace(np.nan, None)

## Main

### Change QTY to Count

In [1471]:
df['Count'] = df['Qty']
df = df.drop('Qty', axis=1)

### Add Edition and Collector Number Columns

In [1473]:
df.insert(len(df.columns), 'Edition', "")
df.insert(len(df.columns), 'Collector Number', "")

### Parse Input DF based on Excel Exceptions Sheet

In [ ]:
for i, row in df.iterrows():
    handle_data_exceptions(row, i, df)

### Correct important columns

In [1474]:
for i, row in df.iterrows():
    df.at[i, 'Name'] = cleanse_card_name(row['Name']).replace("Æ","Ae")
    df.at[i, 'Edition'] = find_correct_set(row['Set'], sets)
    df.at[i, 'Foil'] = "foil" if row['Foil'] else ''
    
    if not row['Collector Number']:
        df.at[i, 'Collector Number'] = row['Number'] if decide_if_land(cleanse_card_name(row['Name'])) else ""

### Drop unnecessary columns

In [1475]:
df = df.drop('Set', axis=1)
df = df.drop('Low Price', axis=1)
df = df.drop('Rarity', axis=1)
df = df.drop('Number', axis=1)

### Finalize column order

In [1476]:
order = ['Count', 'Name', 'Edition', 'Foil', 'Collector Number']
df = df[order]

,Count,Name,Edition,Foil,Collector Number
0,1,Luminous Angel,mrd,,
1,1,Molten Rain,mrd,,
2,1,Honden of Night's Reach,chk,,
3,1,"Nivix, Aerie of the Firemind",gpt,,
4,1,Kithkin Rabble,shm,,


### Save DF to CSV (output folder)

In [1478]:
df.to_csv(OUTPUT_FOLDER_PATH, index=False)